# Solving the Graph Coloring Problem (GCP) with Grover's algortihm and Quantum Approximate Optimization Algorithm (QAOA)

## Introduction for QAOA

The recent improvement of quantum computation hardware has led to the noisy intermediate-scale quantum (NISQ) era. 
In this context, an increasing number of variational quantum algorithms (VQAs) have arisen with the target of solving combinatorial optimization problems (COPs) and achieving quantum advantage.
The main idea of VQAs is to run a parameterized quantum circuit in a model gate-based quantum computer and train these parameters with the aid of classical optimizers.
A promising one is the QAOA, introduced by Farhi et al. in 2014, which is inspired by a Trotterized adiabatic transformation for the adiabatic evolution applied to the gate-based model. 

It is worth mentioning that quantum annealing (QA), as well as VQAs, were conceived as a heuristic approach to solve COPs, especially the ones that can be expressed as binary optimization problems with Boolean variables $x_i \in \{0, \; 1\}$.
Binary optimization involves a wide range of important problems, e. g., social network analysis, portfolio optimization in finance, traffic management and scheduling in transportation, lead optimization in pharmaceutical drug discovery $\ldots$

Taking into account the hardware challange to interact with many qubits, it is mainly aimed to solve Quadratic Unconstrained Binary Optimization (QUBO) problems with 2-local interactions. Its cost function is defined as,
$$
    H_{QUBO} (\vec{x}) = \sum_i h_i x_i + \sum_{ij} Q_{ij} x_i x_j .
$$

However, it could work for solving higher order problems in opposition to the Adiabatic Quantum Optimization (AQO) algorithm.
If the hardware is improved to perform k-local interactions, with $k \geq 3$, high-order polynomial unconstrained binary optimization problems (HOBO) would aim to solve polynomial unconstrained binary optimization (PUBO) problems.

Those might seem strong restrictions, however, lots of problems of interest not only for industry but for academia as well, fall into this category.
A good proof are the Karp's 21 NP-complete problems, with a close connection to Ising spin models as many other NP-hard problems that we can formulate as QUBO problems as Andrew Lucas did in 2014.

### Problem definition and COP encoding on Quantum Computer

In the concerning research, we propose a comparison between Grover's algorithm which and QAOA applied to the vertex coloring problem which is an NP-complete problem contained in the class of GCP.
Graph coloring has been widely studied because it is a subfield highly important in the graph theory field, inspired by the Four Color Problem. 
As it has many applications and relationships with real-life problems, such as scheduling problems including assigning frequencies for radio stations or for a mobile phone network, job shop scheduling, resource allocation, or flight-gate assignments.

When we are dealing with generic COPs of size $N$ usually we have to either maximize or minimize a cost function (also known as objective or loss function), $f: Z(N) \xrightarrow{} \mathbb{R}$, where $Z(N)$ is the set of bit strings with length $N$,
$$
    \min_{z \in S} f(z), \quad S \subseteq Z(N),
$$
where $S$ is the set of feasible bit strings, and we define, $S_{min}$, as the subset of all solutions minimizing $f$. If $S = Z(N)$, it is said that the problem is unconstrained.

To work with a quantum computer we have to express each bit string, $z$, in the computational basis, $\ket{z}$ of the N-qubit space, $\mathcal{H} = \mathbb{C}^{2^N}$. 
This means that each bit, $z_i = 0, \; 1$ of the bit string $z$ is replaced by a spin-1/2 qubit labeled by $\ket{z_i}$, in a way that $\ket{z}$ is the Kronecker product of all the qubit states.
These are eigenstates of the $z$ component of the i-th spin,
$$
    \ket{0} = \begin{pmatrix} 1 \\ 0  \end{pmatrix}, \quad \ket{1} = \begin{pmatrix} 0 \\ 1  \end{pmatrix}
$$
so
$$
    \frac{1}{2} (1 - \sigma_z^{(i)}) \ket{z_i} = z_i \ket{z_i}
$$
where $\sigma_z^{(i)}$ is the corresponding Pauli matrix.
Lastly, the cost Hamiltonian can be constructed as,
$$
    C = \sum_{z \in Z(N)} f(z) \ket{z}\bra{z}.
$$
Therefore, solving the problem is finding an optimal solution in a way that falls to a computational basis state in $S_{min}$ which turns out to be the Hilbert subspace with minimum eigenvalue.

### QAOA

The QAOA is a variational quantum algorithm that iteratively adds $p$ layers of a pair of parameter dependent unitary operators to a given initial quantum state. The first unitary, also called *_mixer_*, depends on an initial Hamiltonian, $B$,
$$
    U_B (\beta) = e^{-i \beta B}
$$
with $\beta \in [0, \pi]$, whilst the second one, known as \textit{phase separator}, depends on the problem Hamiltonian, $C$,
$$
    U_C (\gamma) = e^{-i \gamma C}.
$$
with $\gamma \in [0, 2 \pi]$.
These are executed into a fixed initial state, that is usually selected to be the superposition of all the elements in the computational basis,
$$
    \ket{s} = \frac{1}{\sqrt{2^N}} \sum_{z \in Z(N)} \ket{z}
$$
which is the superposition of all possible solutions (for unconstrained problems).

From these unitaries, a parametrized trial state is constructed as,
$$ 
    \ket{\vec{\beta}, \vec{\gamma}} = V(\vec{\beta}, \vec{\gamma}) \ket{s} = \left( \prod_{q=1}^p U_B(\beta_p) U_C (\gamma_q) \right) \ket{s}.
$$

The parameters of the trial state are optimized by minimizing the value of the following eqaution, i.e. the expected value of the cost Hamiltonian, with a classical optimizer.
$$
    F_p (\vec{\beta}, \vec{\gamma}) = \bra{\vec{\beta}, \vec{\gamma}} C \ket{\vec{\beta}, \vec{\gamma}}.
$$
Once this process converges to the final state, $\ket{\vec{\beta_{out}}, \vec{\gamma_{out}}}$, one can obtain a solution to the problem by measuring the state in the computational basis. If properly optimized, the solution improves with $p$, and in its limit to $\infty$, it is guaranteed to reach the solution with overlap 1 \cite{qaoa, convergence_qaoa}.

\section{Vertex Coloring Problem}

Solving the vertex coloring problem consists in checking if a considered graph can have its vertex colored such that each edge is not connecting two vertices with the same color. It is expressed mathematically in graph theory as the proper mapping $f: V(G) \xrightarrow{} \mathbb{N}$ such that,
$$
    \forall_{v_i, v_j \in V(G), i \neq j} \exists (e_i, \; e_j) \implies f(i) \neq f(j).
$$

As an example, solving the problem for a fully connected graph is trivial, with the number of colors required $n$ being equal to the number of vertices $v$.
%A much more interesting case is the planar graph because it can be proved that there is always a coloring for $n \geq 4$.
%This is known as the Four Color Theorem which is one of the problems that originated GCP class and made it emerge as one of the most studied classes of problems in graph theory. 
Note that, any solution is degenerate due to any of its permutation of colors also being one ($n!$ in total).
%If a given graph $G$ is using at most $n$ colors it is called a (proper) $n$-coloring.

%An important feature that characterizes a given graph, $G$, is the minimum number of colors needed to meet the vertex coloring problem constraints, and it is called the chromatic number, $\chi (G)$.
%Determining $\chi (G)$ is an NP-hard problem because it means adding a constraint of the number of colors and then the cost function has to be minimized, in contrast with what we will do which is fulfilling the cost function constraints.

### QUBO formulation and implementation
Consider the undirected graph $G = (V, E)$ and several colors $n$, where the number of vertices and edges is given by $|V|$ and $|E|$, respectively. Next, define the binary variable, $x_{v, i}$, which is 1 if vertex $v$ is colored with the color $i$, and 0 otherwise. The total number of binary variables (qubits) is $|V|\times n$. Then \eqref{qubo} is s.t. its ground state (with energy 0) encodes the solution of the problem. For any other candidate that breaks the constraints, its energy is penalized.
$$
    H = A \sum_v \left( 1- \sum_{i=1}^n x_{v, i}  \right)^2 + A \sum_{(uv) \in E} \; \sum_{i=1}^n x_{u, i} x_{v, i},
$$
The first term ensures that each vertex has only one color assigned, and the second term penalizes two adjacent vertices $x_{v, i}$ and $x_{u, i}$, colored with the same color.

The cost function is translated to a quantum Hamiltonian by mapping each binary variable to $\mathcal{\hat{Z}}_i = \frac{\mathcal{I} - \sigma^{z}_i}{2}$,  with $\sigma^{z}_i$, the Z Pauli matrix acting on the $i$th qubit in a Hilbert space of $N$ qubits. This transformation has eigenvectors $\ket{0}$ and $\ket{1}$ with eigenvalues 0 and 1, respectively as desired.

Then for both QAA and QAOA, the driving or mixer Hamiltonian, that has $\ket{s}$ as a ground state with energy 0, is,
$$
    B = H_0 = \sum_{i=1}^N \frac{\mathcal{I}_i - \sigma^x_i}{2}.
$$
However, to perform a fair comparison we keep the original formulation with the mixer equal to the initial Hamiltonian in QAA. 
Moreover, a recently published article, \cite{Golden_mixers}, has shown that more complex mixers, that reduce the searched Hilbert Space, perform significantly worse as the dimension of the problem grows.